In [92]:
import requests as rq
from bs4 import BeautifulSoup as bs
from bs4.element import Comment
import codecs
import collections
import pandas as pd
import numpy as np
import scipy as scp
import re
import os

In [93]:
# função utilizada para baixar uma página html e savar em name
def getPage(url, name, codec):
    
    if not os.path.isfile(name):
        page = rq.get(url)
        f = codecs.open(name, 'w', codec)
        f.write(page.text)
        f.close

In [139]:
# utilizado para filtrar apenas of elementos html que são visiveis para o usuário
def tag_visible(element):
    
    if element.parent.name in ['style', 'script', 'head', 'title', 'meta', '[document]']:
        return False
    if isinstance(element, Comment):
        return False
    return True

# utilizado para pegar apenas o texto da página html
def getText(arquivo):        
    bsFile = bs(open(arquivo),"html.parser")
    texts = bsFile.findAll(text=True)
    visible_texts = filter(tag_visible, texts)  
    words = [t.strip() for t in visible_texts]
    words = filter(lambda x: len(x)>0, words)
    text = " ".join(words)
    return text

# utilizado para pegar as palavras que aparecem em uma página html (apenas páginas salvas localmente)
def getUniqueWords(arquivo):
    text = getText(arquivo)
    words = text.split(" ")
    uniqueWords = collections.Counter(words)
    return [word for word in uniqueWords.keys()]

def getAllUniqueWords(diretorio):
    files = os.listdir(diretorio)
    files = filter(lambda x: x[-5:].lower() == ".html", files)
    allWords = []
    for index, f in enumerate(files):
        allWords = allWords + getUniqueWords(diretorio + "/" + f)
        print index, len(allWords)
        
    uniqueWords = collections.Counter(allWords)
    return [word for word in uniqueWords.keys()]
    
uniqueWords = getAllUniqueWords("sites")
f = open("uniqueWords.txt", "w")
for word in uniqueWords[:-1]:
    f.write(word + ",")
f.write(uniqueWords[-1] + "\n")
f.close()

0 564
1 882
2 1230
3 1499
4 2795
5 3425
6 3988
7 5457
8 5540
9 6154
10 6751
11 6834
12 7510
13 8325
14 10701
15 10704
16 11312
17 11850
18 12194
19 12436
20 12519
21 13372
22 14053
23 14532
24 15283
25 15912
26 17288
27 17788
28 18349
29 19173
30 19444
31 19764
32 20397
33 21025
34 21641
35 22211
36 22470
37 22981
38 23573
39 24145
40 24228
41 24311
42 24394
43 24913
44 25476
45 25559
46 26189
47 26461
48 26799
49 26882
50 27424
51 28281
52 28797
53 28880
54 29462
55 29545
56 30043
57 30627
58 31307
59 31786
60 32698
61 33260
62 34396
63 34973
64 35484
65 36095
66 36178
67 36261
68 36895
69 37458
70 37616
71 38123
72 38755
73 39025
74 39594
75 39940
76 40603
77 40686
78 41234
79 41317
80 41926
81 42167
82 42837
83 43610
84 43925
85 44239
86 44773
87 45403
88 45900
89 49227
90 49310
91 49763
92 50283
93 50762
94 51348
95 51690
96 51970
97 57511
98 58371
99 58542
100 59168
101 59687
102 60349
103 61343
104 62036
105 62718
106 62801
107 63282
108 63847
109 64407
110 65146
111 65456
112 65

UnicodeEncodeError: 'ascii' codec can't encode character u'\xf3' in position 0: ordinal not in range(128)

In [140]:
f = codecs.open("uniqueWords.txt", "w", "utf-8")
for word in uniqueWords[:-1]:
    f.write(word + ",")
f.write(uniqueWords[-1])
f.close

<function close>

In [164]:
# removendo tabs e \n \r \t
fr = codecs.open("uniqueWords.txt", "r", "utf-8")
fw = codecs.open("uniqueWords2.txt", "w", "utf-8")
frwords = fr.read().split(",")
for frw in frwords[:-1]:
    frw = ",".join(frw.split("\r"))
    frw = ",".join(frw.split("\t"))
    frw = ",".join(frw.split("\n"))
    frw = ",".join(frw.split(" "))
    fw.write(frw + ",")

frw = frwords[-1]    
frw = ",".join(frw.split("\r"))
frw = ",".join(frw.split("\t"))
frw = ",".join(frw.split("\n"))
frw = ",".join(frw.split(" "))
fw.write(frw)
fr.close()

In [165]:
# removendo palavras vazias
fr = codecs.open("uniqueWords2.txt", "r", "utf-8")
fw = codecs.open("uniqueWords3.txt", "w", "utf-8")
frwords = fr.read().split(",")
for frw in frwords[:-1]:
    if len(frw) > 0:
        fw.write(frw + ",")

if len(frwords[-1]) > 0:
    fw.write(frwords[-1]) 
fr.close()
fw.close()

In [163]:
# removendo palavras vazias
fr = codecs.open("uniqueWords3.txt", "r", "utf-8")
print len(fr.read().split(","))
fr.close()

19400


In [60]:
# utilizado para ler o arquivo com as páginas utilizadas no projeto
data_set = pd.read_csv("sites.csv")

In [95]:
# data_set.describe()

In [94]:
# utizado para baixar as páginas html que foram utilizadas no projeto

padrao = '((www\.)?[a-zA-Z0-9]+\.[a-zA-Z0-9]+(\.[a-zA-Z0-9]+)*)'
for i, site in enumerate(data_set.url):
    name = str(i) + re.findall(padrao, site)[0][0] + ".html"
    getPage(url=site, name="sites/" + name, codec="utf-8")
    print "finalizou " + name

In [167]:
# fazendo base
fr = codecs.open("uniqueWords3.txt", "r", "utf-8")
frwords = fr.read().split(",")
print len(frwords)
fr.close()

19399


In [ ]:
# def getAllUniqueWords(diretorio):
# fr = codecs.open("uniqueWords3.txt", "r", "utf-8")
# frwords = fr.read().split(",")
# print len(frwords)
# fr.close()

files = os.listdir("sites")
files = filter(lambda x: x[-5:].lower() == ".html", files)
for index, f in enumerate(files):
    allWords = getUniqueWords(diretorio + "/" + f)

    for frw in allWords:
        frw = ",".join(frw.split("\r"))
        frw = ",".join(frw.split("\t"))
        frw = ",".join(frw.split("\n"))
        frw = ",".join(frw.split(" "))
    
    for frw in frwords[:-1]:
        if len(frw) > 0:
            fw.write(frw + ",")

uniqueWords = collections.Counter(allWords)
# return [word for word in uniqueWords.keys()]